In [1]:
#modules used in your code
import numpy as np
import matplotlib.pyplot as plt

In [1]:
#modules used in your code
import numpy as np
import matplotlib.pyplot as plt
def shortest_path(costs):
    """
    This function takes an array of costs and finds a shortest path from the 
    top to the bottom of the array which minimizes the total costs along the
    path. The path should not shift more than 1 location left or right between
    successive rows.
    
    In other words, the returned path is one that minimizes the total cost:
        
        total_cost = costs[0,path[0]] + costs[1,path[1]] + costs[2,path[2]] + ...

    subject to the constraint that:
    
        abs(path[i]-path[i+1])<=1
    
    Parameters
    ----------
    costs : 2D float array of shape HxW
         An array of cost values with W>=3
    
    Returns
    -------
    path : 1D array of length H
        indices of a vertical path.  path[i] contains the column index of 
        the path for each row i. 
    """

    nrows = costs.shape[0]
    ncols = costs.shape[1]
    
    # to keep the implementation simple, we will refuse to handle
    # the boundary case where the cost array is very narrow.
    assert(ncols>=3)
    print()
    
    
    #init value
    a= np.full((nrows,ncols),0.00)
    a[0][0]=costs[0][0]
    l1=[]
    localmin= (999,0)
    for i in range(0, ncols):
        
        a[0][i] = costs[0][i]
    
    #print(a)
    
    #perform the dp, for each position, it can come from three j points, I also do boudary check to some j will not be collected
    for i in range(1, nrows):
        for j in range(0, ncols):
            
            if(j==ncols-1):
                x1=a[i-1][ncols-1] + costs[i][ncols-1]
                x2=a[i-1][ncols-2] + costs[i][ncols-1]
                #print(x1,x2)
                a[i][ncols-1] = min(x1,x2)
           
            
            
            elif j>0:
                x1=a[i-1][j-1] + costs[i][j]
                x2=a[i-1][j] + costs[i][j]
                x3=a[i-1][j+1] + costs[i][j]
                
                a[i][j] = min(x1,x2,x3)
        
                
                
            elif j==0:
                x2=a[i-1][j] + costs[i][j]
                x3=a[i-1][j+1] + costs[i][j]
                a[i][j] = min(x2,x3)
            
    err = 1e-10
    
    #trace back the position store it in the l1 list
    count=0
    
    spath = min(a[nrows-1])
    ind1=nrows-1
    ind2=0
    
    #get the bottom position with smallest cost
    for i in range(ncols):
        
        if a[ind1][i] == spath:
            break
        count+=1
    ind2 =count
    l1.append(ind2)
    lastj=ind2
    
    
    for i in range(nrows-2,-1,-1):
        for j in range(ncols):
            #print(spath- costs[ind1][ind2],"dangqian  ",a[i][j])
            diff = a[i][j]  - (spath- costs[ind1][ind2])
            if abs(diff)<err :
                l1.append(j)
                #print("jiaru le",j)
                lastj =j
                spath = spath-costs[ind1][ind2]
                ind1 = i
                ind2 = j
                
        #print("end zhelun")
    l1.reverse()
    #print(l1)
    path = np.array(l1)
    return path

In [28]:
def stitch(left_image, right_image, overlap):
    """
    This function takes a pair of images with a specified overlap and stitches them
    togther by finding a minimal cost seam in the overlap region.
    
    Parameters
    ----------
    left_image : 2D float array of shape HxW1
        Left image to stitch 
    
    right_image : 2D float array of shape HxW2
        Right image to stitch 

    overlap : int
        Width of the overlap zone between left and right image
        
    Returns
    -------
    stitched : 2D float array of shape Hx(W1+W2-overlap)
        The resulting stitched image
    """
    
    # inputs should be the same height
    assert(left_image.shape[0]==right_image.shape[0])
    assert(overlap>=3)
    
    
    
    leftrow=left_image.shape[0]
    leftcol=left_image.shape[1]
    
    rightrow=right_image.shape[0]
    rightcol=right_image.shape[1]
    # caculate the overlap region 
    overlap1=left_image[0:leftrow,(leftcol-overlap):leftcol]
    overlap2=right_image[0:leftrow,0:overlap]
    
    
    #caculate spath 
    spath = shortest_path(abs(overlap1-overlap2))
    

   #construct the stitched array since height is fixed and width is w1+w2 -overlap
    if leftrow == rightrow:
        stitched = np.zeros((leftrow,leftcol+rightcol-overlap))
    
    
    
    finalrownum = stitched.shape[0]
    finalcolnum = stitched.shape[1]
    for i in range(finalrownum):
        spathi = spath[i]

        
        count=0
        leftlimit = leftcol+spathi-overlap
        #make sure within colnum
        while count < finalcolnum:
            #print(finalcolnum)
            #left part
            if count <= leftlimit:
                #print(count,"J  ",spathi)
                stitched[i,count]=left_image[i,count]
                count+=1
                #print(spathi,"path")
            #right part
            else:
                    stitched[i,count]=right_image[i,spathi]
                    count+=1
                    spathi+=1
                    #print(spathi,"path")
                
        
    
    
    
    assert(stitched.shape[0]==left_image.shape[0])
    assert(stitched.shape[1]==(left_image.shape[1]+right_image.shape[1]-overlap))
    
    return stitched
    
    

In [10]:
def synth_quilt(tile_map,tiledb,tilesize,overlap):
  
    """
    This function takes as input an array indicating the set of texture tiles 
    to use at each location, an array containing the database of available texture 
    tiles, tilesize and overlap parameters, and synthesizes the output texture by 
    stitching together the tiles

    
    Parameters
    ----------
    tile_map : 2D array of int
        Array storing the indices of which tiles to paste down at each output location 
    
    tiledb : 2D array of int 
        Collection of sample tiles to select from. The array is of size ntiles x npixels 
        where each tile image is stored in vectorized form as a row of the array.
        
    tilesize : (int,int)
        Size of a tile in pixels
        
    overlap : int
        Amount of overlap between tiles
        
    Returns
    -------
    output : 2D float array
        The resulting synthesized texture of size
    """
    
    # determine output size based on overlap and tile size
    outh = (tilesize[0]-overlap)*tile_map.shape[0] + overlap
    outw = (tilesize[1]-overlap)*tile_map.shape[1] + overlap
    output = np.zeros((outh,outw))

    
    
    
    rowlist = []

    rownum= tile_map.shape[0]
    colnum = tile_map.shape[1]
    

        
    for i in range(rownum):

        #use this location to store value and update them 
        store = output[0:tilesize[1],0:tilesize[1]] 
        # store the value of beginning tile_image

        tile_vec = tiledb[tile_map[i,0]]
        tile_image = np.reshape(tile_vec,tilesize)
        store = tile_image



        for j in range(1,colnum):
            #  update the value of rest tile_image in the row
            tile_vec = tiledb[tile_map[i,j]]
            tile_image2 = np.reshape(tile_vec,tilesize)
            store = stitch(store,tile_image2,overlap)

        rowlist.append(store)
    #print(rowlist)
    #print(rowlist)


    tempout= np.zeros(tilesize)
    count = 0
    for i in range(len(rowlist)):
        if count == 0:
            #take out the first row data and transpose
            tempout = rowlist[i].T
            count+=1
        else:
            
            #update the value with rest row data in list
            tempout = stitch(tempout, rowlist[i].T, overlap)
            count+=1

        
        
    
    return tempout.T



In [30]:
#skimage is only needed for sample tiles code provided below
#you should not use it elsewhere in your own code
import skimage as ski 

def sample_tiles(image,tilesize,randomize=True):
    """
    This function generates a library of tiles of a specified size from a given source image
       
    Parameters
    ----------
    image : float array of shape HxW
        Input image 
            
    tilesize : (int,int)
        Dimensions of the tiles in pixels
        
        
    Returns
    -------
    tiles : float array of shape  numtiles x numpixels
        The library of tiles stored as vectors where npixels is the 
        product of the tile height and width     
    """
    
    tiles = ski.util.view_as_windows(image,tilesize)
    ntiles = tiles.shape[0]*tiles.shape[1]
    npix = tiles.shape[2]*tiles.shape[3]
    assert(npix==tilesize[0]*tilesize[1])
    
    print("library has ntiles = ",ntiles,"each with npix = ",npix)
    
    tiles = tiles.reshape((ntiles,npix)) 
    
    # randomize tile order
    if randomize:
        tiles = tiles[np.random.permutation(ntiles),:]
    
    return tiles


def topkmatch(tilestrip,dbstrips,k):
    """
    This function finds the top k candidate matches in dbstrips that
    are most similar to the provided tile strip.
      
    Parameters
    ----------
    tilestrip : 1D float array of length npixels
        Grayscale values of the query strip 
    
    dbstrips : 2D float array of size npixels x numtiles
        Array containing brightness values of numtiles strips in the database 
        to match to the npixels brightness values in tilestrip
                
    k : int
        Number of top candidate matches to sample from
        
    Returns
    -------
    matches : list of ints of length k
        The indices of the k top matching tiles   
    """
    assert(k>0)
    assert(dbstrips.shape[0]>k)
    error = (dbstrips-tilestrip)
    ssd = np.sum(error*error,axis=1)
    ind = np.argsort(ssd)
    matches = ind[0:k]
    return matches


def quilt_demo(sample_image, ntilesout=(10,20), tilesize=(30,30), overlap=5, k=5):
    """
    This function takes an image and quilting parameters and synthesizes a
    new texture image by stitching together sampled tiles from the source image.

     
    Parameters
    ----------
    sample_image : 2D float array
        Grayscale image containing sample texture 
    
    ntilesout : list of int
        Dimensions of output in tiles,  e.g. (3,4)
        
    tilesize : int
        Size of the square tile in pixels
        
    overlap : int
        Amount of overlap between tiles
        
    k : int
        Number of top candidate matches to sample from
        
    Returns
    -------
    img : list of int of length K
        The resulting synthesized texture of size           
    """
    
    # generate database of tiles from sample
    tiledb = sample_tiles(sample_image,tilesize)
    # number of tiles in the database
    nsampletiles = tiledb.shape[0]
    
    if (nsampletiles<k):
        print("Error: tile database is not big enough!")

    # generate indices of the different tile strips so we can easily
    # extract the left, right, top or bottom overlap strip from a tile
    i,j = np.mgrid[0:tilesize[0],0:tilesize[1]]
    top_ind = np.ravel_multi_index(np.where(i<overlap),tilesize)
    bottom_ind = np.ravel_multi_index(np.where(i>=tilesize[0]-overlap),tilesize)
    left_ind = np.ravel_multi_index(np.where(j<overlap),tilesize)
    right_ind = np.ravel_multi_index(np.where(j>=tilesize[1]-overlap),tilesize)

    # initialize an array to store which tile will be placed
    # in each location in the output image
    tile_map = np.zeros(ntilesout,'int')
    
    print('row:')
    for i in range(ntilesout[0]):
        print(i)
        for j in range(ntilesout[1]):
            
            if (i==0)&(j==0):                  # first row first tile
                matches = np.zeros(k) #range(nsampletiles)
            
            elif (i==0):                       # first row (but not first tile)
                left_tile = tile_map[i,j-1]
                tilestrip = tiledb[left_tile,right_ind]
                dbstrips = tiledb[:,left_ind]
                matches = topkmatch(tilestrip,dbstrips,k)
                
            elif (j==0):                       # first column (but not first row)
                above_tile = tile_map[i-1,j]
                tilestrip = tiledb[above_tile,bottom_ind]
                dbstrips = tiledb[:,top_ind] 
                matches = topkmatch(tilestrip,dbstrips,k)
                
            else:                              # neigbors above and to the left
                left_tile = tile_map[i,j-1]
                tilestrip_1 = tiledb[left_tile,right_ind]
                dbstrips_1 = tiledb[:,left_ind]
                above_tile = tile_map[i-1,j]
                tilestrip_2 = tiledb[above_tile,bottom_ind]
                dbstrips_2 = tiledb[:,top_ind]
                # concatenate the two strips
                tilestrip = np.concatenate((tilestrip_1,tilestrip_2))
                dbstrips = np.concatenate((dbstrips_1,dbstrips_2),axis=1)
                matches = topkmatch(tilestrip,dbstrips,k)
                
            #choose one of the k matches at random
            tile_map[i,j] = matches[np.random.randint(0,k)]
    
    output = synth_quilt(tile_map,tiledb,tilesize,overlap)
    
    return output
